In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import helpers.processing_helpers as ph

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import make_scorer


In [2]:
df_dev = pd.read_csv("./dataset/development.csv")

In [3]:
noise_indexes = [0,7,12,15,16,17]
acc_idxs = [1,2,3,4,5,6,8,9,10,11,13,14]
features = ["pmax", "negpmax", 'area', 'tmax', 'rms']

sensors_removed = df_dev.drop(columns=ph.get_column_names(features, noise_indexes))
sensors_removed = sensors_removed.drop(columns=ph.get_column_names(['tmax', 'rms'], acc_idxs))

In [4]:
negpmax_clms = sensors_removed[[f'negpmax[{i}]' for i in acc_idxs]]
df_above_zero = negpmax_clms[(negpmax_clms > 0).any(axis=1)]
df_zero = sensors_removed.drop(df_above_zero.index.values)

In [5]:
X_train, y_train = ph.insert_zeros(df_zero, acc_idxs, 17)

In [6]:
reg = RandomForestRegressor(max_features='sqrt', n_estimators=400, max_depth=40, verbose=1)
reg.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:  2.9min
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed: 11.9min


RandomForestRegressor(max_depth=40, max_features='sqrt', n_estimators=400,
                      verbose=1)

In [7]:
df_eval = df_dev = pd.read_csv("./dataset/evaluation.csv")

In [8]:
noise_indexes = [0,7,12,15,16,17]
acc_idxs = [1,2,3,4,5,6,8,9,10,11,13,14]
features = ["pmax", "negpmax", 'area', 'tmax', 'rms']

sensors_removed = df_eval.drop(columns=ph.get_column_names(features, noise_indexes))
sensors_removed = sensors_removed.drop(columns=ph.get_column_names(['tmax', 'rms'], acc_idxs))

In [9]:
X_train, _ = ph.insert_zeros(sensors_removed, acc_idxs, 17, mode='evaluation')

In [10]:
y_pred = reg.predict(X_train)

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    7.8s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:   34.2s


In [11]:
submission = ph.create_submission(y_pred)

In [12]:
submission.to_csv('./submissions/submission_6.csv', index=False)
# 4.814